<a href="https://colab.research.google.com/github/gatmaitan/TD-Ameritrade-API-Python-Wrapper/blob/main/Strat_Mean_Reversion_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link : [link text](https://tradewithpython.com/generating-buy-sell-signals-using-python)

In [102]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
import pandas as pd
import pandas_ta as ta
from datetime import date

In [104]:
#get data
#key = DN2DFKCJ6ZLV472M
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=aapl&apikey=DN2DFKCJ6ZLV472M&datatype=csv'

In [105]:
data = pd.read_csv(url)
print(df)

     timestamp     open     high      low   close  adjusted_close    volume  \
0   2023-01-20  135.280  138.020  134.220  137.87      137.870000  80223626   
1   2023-01-19  134.080  136.250  133.770  135.27      135.270000  58280413   
2   2023-01-18  136.815  138.610  135.030  135.21      135.210000  69672800   
3   2023-01-17  134.830  137.290  134.130  135.94      135.940000  63646627   
4   2023-01-13  132.030  134.920  131.660  134.76      134.760000  57809719   
..         ...      ...      ...      ...     ...             ...       ...   
95  2022-09-02  159.750  160.362  154.965  155.81      155.355057  76957768   
96  2022-09-01  156.640  158.420  154.670  157.96      157.498779  74229896   
97  2022-08-31  160.305  160.580  157.140  157.22      156.760940  87991091   
98  2022-08-30  162.130  162.560  157.720  158.91      158.446005  77906197   
99  2022-08-29  161.145  162.900  159.820  161.38      160.908793  72724452   

    dividend_amount  split_coefficient  
0         

# Simple Moving Averages

This code is calculating simple moving averages (SMA) for two different periods (30 and 100 days) for the closing prices of a financial instrument, and then using those SMAs to generate buy and sell signals.

The first two lines of code calculate the 30-day and 100-day SMAs for the "close" column in the data and store them as new columns in the dataframe named "SMA 30" and "SMA 100" respectively.

The function "buy_sell" then takes the dataframe as an input and initializes empty lists "signalBuy" and "signalSell" to store the buy and sell signals respectively, and a boolean variable "position" to keep track of the current position (long or short).

The function then iterates over each element of the dataframe, and checks whether the 30-day SMA is greater than the 100-day SMA. If it is, and the current position is short, it generates a buy signal by appending the adjusted closing price to the "signalBuy" list and appending NaN to the "signalSell" list. It also changes the position to long. If the current position is already long, it does not generate any signal.

If the 30-day SMA is less than the 100-day SMA, it generates a sell signal and changes the position to short if the current position is long. If the current position is already short, it does not generate any signal.

If the 30-day SMA is equal to the 100-day SMA, it does not generate any signal.

Finally, it returns a pandas series containing the "signalBuy" and "signalSell" lists.

In [106]:
data['SMA 30'] = ta.sma(data['close'],30)
data['SMA 100'] = ta.sma(data['close'],100)
#SMA BUY SELL
#Function for buy and sell signal
def buy_sell(data):
    signalBuy = []
    signalSell = []
    position = False 

    for i in range(len(data)):
        if data['SMA 30'][i] > data['SMA 100'][i]:
            if position == False :
                signalBuy.append(data['adjusted_close'][i])
                signalSell.append(np.nan)
                position = True
            else:
                signalBuy.append(np.nan)
                signalSell.append(np.nan)
        elif data['SMA 30'][i] < data['SMA 100'][i]:
            if position == True:
                signalBuy.append(np.nan)
                signalSell.append(data['adjusted_close'][i])
                position = False
            else:
                signalBuy.append(np.nan)
                signalSell.append(np.nan)
        else:
            signalBuy.append(np.nan)
            signalSell.append(np.nan)
    return pd.Series([signalBuy, signalSell])


# MACD


This code is implementing a trading strategy that uses the moving average convergence divergence (MACD) indicator to generate buy and sell signals. The MACD is a momentum and trend-following indicator that uses the difference between two exponential moving averages to identify changes in direction and momentum.

The function "MACD_Strategy" takes a dataframe "df" and a risk variable as inputs. It initializes empty lists "MACD_Buy" and "MACD_Sell" to store the buy and sell signals respectively, and a boolean variable "position" to keep track of the current position (long or short).

The function then iterates over each element of the dataframe, and checks whether the MACD line is greater than the signal line. If it is, it generates a buy signal and changes the position to long if the current position is short. If the current position is already long, it does not generate any signal.

If the MACD line is less than the signal line, it generates a sell signal and changes the position to short if the current position is long. If the current position is already short, it does not generate any signal.

It also checks for the stop-loss condition, if the position is long and the current adjusted close is less than the previous adjusted close * (1-risk) then it generates a sell signal, changes the position to short.

Finally, it adds the "MACD_Buy" and "MACD_Sell" lists as new columns in the dataframe, named "MACD_Buy_Signal_price" and "MACD_Sell_Signal_price" respectively.

In [107]:
macd = ta.macd(data['close'])

In [108]:
def MACD_Strategy(df, risk):
    MACD_Buy=[]
    MACD_Sell=[]
    position=False

    for i in range(0, len(df)):
        if df['MACD_12_26_9'][i] > df['MACDs_12_26_9'][i] :
            MACD_Sell.append(np.nan)
            if position ==False:
                MACD_Buy.append(df['Adj Close'][i])
                position=True
            else:
                MACD_Buy.append(np.nan)
        elif df['MACD_12_26_9'][i] < df['MACDs_12_26_9'][i] :
            MACD_Buy.append(np.nan)
            if position == True:
                MACD_Sell.append(df['Adj Close'][i])
                position=False
            else:
                MACD_Sell.append(np.nan)
        elif position == True and df['Adj Close'][i] < MACD_Buy[-1] * (1 - risk):
            MACD_Sell.append(df["Adj Close"][i])
            MACD_Buy.append(np.nan)
            position = False
        elif position == True and df['Adj Close'][i] < df['Adj Close'][i - 1] * (1 - risk):
            MACD_Sell.append(df["Adj Close"][i])
            MACD_Buy.append(np.nan)
            position = False
        else:
            MACD_Buy.append(np.nan)
            MACD_Sell.append(np.nan)

    data['MACD_Buy_Signal_price'] = MACD_Buy
    data['MACD_Sell_Signal_price'] = MACD_Sell


In [109]:
data = pd.concat([data, macd], axis=1).reindex(data.index)

# Bollinger Bands

This code defines a function called "bb_strategy" that takes in a dataframe called "data" as an input. Within the function, two lists called "bbBuy" and "bbSell" are created, as well as a variable "position" that is initialized as "False".

The function uses the "ta" library to calculate the Bollinger Bands for the "Adj Close" column of the dataframe, with a length of 20 and a standard deviation of 2. It then concatenates the resulting Bollinger Bands data with the original dataframe and reindexes it to match the original dataframe's index.

The function then uses a for loop to iterate through the rows of the dataframe. For each iteration, it checks whether the value of "Adj Close" for that row is less than the value of the lower Bollinger Band "BBL_20_2.0" for that row. If it is, and the position variable is "False", it appends the value of "Adj Close" for that row to the "bbBuy" list, and appends a "NaN" value to the "bbSell" list. It also sets the position variable to "True". If the position variable is already "True", it appends "NaN" values to both lists.

If the value of "Adj Close" is greater than the value of the upper Bollinger Band "BBU_20_2.0" for that row, it checks if the position variable is "True". If it is, it appends a "NaN" value to the "bbBuy" list, and the value of "Adj Close" for that row to the "bbSell" list, and sets the position variable to "False". If the position variable is already "False", it appends "NaN" values to both lists.

If the value of "Adj Close" is between the value of the lower Bollinger Band and upper Bollinger Band for that row, it appends "NaN" values to both lists.

Finally, the function creates two new columns in the dataframe named "bb_Buy_Signal_price" and "bb_Sell_Signal_price" and assigns the "bbBuy" and "bbSell" lists to them respectively.

The function then returns the modified dataframe.

This function is similar to the one before, but instead of using moving average indicators, it uses Bollinger Bands indicators to buy or sell. When the close price of a stock falls below the lower Bollinger Band it is considered as a buy signal, whereas if it moves above the upper Bollinger Band it is considered as a sell signal.

In [110]:
def bb_strategy(data):
    bbBuy = []
    bbSell = []
    position = False
    bb = ta.bbands(data['adjusted_close'], length=20,std=2)
    data = pd.concat([data, bb], axis=1).reindex(data.index)

    for i in range(len(data)):
        if data['adjusted_close'][i] < data['BBL_20_2.0'][i]:
            if position == False :
                bbBuy.append(data['adjusted_close'][i])
                bbSell.append(np.nan)
                position = True
            else:
                bbBuy.append(np.nan)
                bbSell.append(np.nan)
        elif data['adjusted_close'][i] > data['BBU_20_2.0'][i]:
            if position == True:
                bbBuy.append(np.nan)
                bbSell.append(data['adjusted_close'][i])
                position = False 
            else:
                bbBuy.append(np.nan)
                bbSell.append(np.nan)
        else :
            bbBuy.append(np.nan)
            bbSell.append(np.nan)

    data['bb_Buy_Signal_price'] = bbBuy
    data['bb_Sell_Signal_price'] = bbSell

    return data


In [111]:
#storing the function
data = bb_strategy(data)
print(data)

     timestamp     open     high      low   close  adjusted_close    volume  \
0   2023-01-20  135.280  138.020  134.220  137.87      137.870000  80223626   
1   2023-01-19  134.080  136.250  133.770  135.27      135.270000  58280413   
2   2023-01-18  136.815  138.610  135.030  135.21      135.210000  69672800   
3   2023-01-17  134.830  137.290  134.130  135.94      135.940000  63646627   
4   2023-01-13  132.030  134.920  131.660  134.76      134.760000  57809719   
..         ...      ...      ...      ...     ...             ...       ...   
95  2022-09-02  159.750  160.362  154.965  155.81      155.355057  76957768   
96  2022-09-01  156.640  158.420  154.670  157.96      157.498779  74229896   
97  2022-08-31  160.305  160.580  157.140  157.22      156.760940  87991091   
98  2022-08-30  162.130  162.560  157.720  158.91      158.446005  77906197   
99  2022-08-29  161.145  162.900  159.820  161.38      160.908793  72724452   

    dividend_amount  split_coefficient      SMA 30 

In [112]:
#print(data['bb_Buy_Signal_price'].dropna())
print(data['bb_Buy_Signal_price'])

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
      ..
95   NaN
96   NaN
97   NaN
98   NaN
99   NaN
Name: bb_Buy_Signal_price, Length: 100, dtype: float64


# RSI

In [113]:
# calculate RSI
RSI = ta.rsi(data['adjusted_close'], length = 14)

# ADX

In [114]:
getADX = ta.adx(data['high'],data['low'],data['adjusted_close'])
print(getADX)

       ADX_14     DMP_14     DMN_14
0         NaN        NaN        NaN
1         NaN        NaN        NaN
2         NaN        NaN        NaN
3         NaN        NaN        NaN
4         NaN        NaN        NaN
..        ...        ...        ...
95  18.989451  26.236896  15.571675
96  19.371890  24.979633  15.202454
97  20.099924  26.688692  14.515304
98  21.096759  27.254026  13.414531
99  22.077066  26.095955  12.622756

[100 rows x 3 columns]
